In [14]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas_datareader.data as web # requires v0.6.0 or later
from datetime import datetime
import pandas as pd
import plotly.graph_objs as go

app = dash.Dash()
ciss = pd.read_csv('etf_from_2011.csv')
hf = pd.read_csv('HFdata.csv')
#Index dataset by 'Date' column.
ciss.set_index('Date', inplace=True)
hf.set_index('Date',inplace=True)

In [13]:
options = [{'label': i, 'value': i} for i in ciss.columns]
#options.append({'label':'{}'.format('SPY'), 'value':'SPY'})
#[{'label': i, 'value': i} for i in ciss.columns]
app.layout = html.Div([
    html.H1('ETF data'),
    html.Div([
        html.H3('Select ETF:', style={'paddingRight':'30px'}),
        # replace dcc.Input with dcc.Options, set options=options
        dcc.Dropdown(
            id='my_ticker_symbol',
            options=options,
            value=['SPY'],
            #multi=True
        )
    # widen the Div to fit multiple inputs
    ], style={'display':'inline-block', 'verticalAlign':'top', 'width':'30%'}),
    html.Div([
    html.H2('HF data'),
    html.Div([
        html.H3('Select Hedge Fund:', style={'paddingRight':'30px'}),
        # replace dcc.Input with dcc.Options, set options=options
        dcc.Dropdown(
            id='my_fund_symbol',
            options=options,
            value=['FX'],
            #multi=True
        )
    # widen the Div to fit multiple inputs
    ], style={'display':'inline-block', 'verticalAlign':'top', 'width':'30%'}),
    html.Div([
        html.H3('Select start and end dates:'),
        dcc.DatePickerRange(
            id='my_date_picker',
            min_date_allowed=datetime(2011, 1, 1),
            max_date_allowed=datetime(2019, 1, 2),
            start_date=str(datetime(2011, 1, 1)),
            end_date=str(datetime(2019, 1, 2))
        )
    ], style={'display':'inline-block'}),
    html.Div([
        html.Button(
            id='submit-button',
            n_clicks=0,
            children='Submit',
            style={'fontSize':24, 'marginLeft':'30px'}
        ),
    ], style={'display':'inline-block'}),
    dcc.Graph(
        id='my_graph',
        figure={
            'data': [
                {'x': [1,2], 'y':[3,1]}
            ]
        }
    )
])
@app.callback(
    Output('my_graph', 'figure'),
    [Input('submit-button', 'n_clicks')],
    [State('my_ticker_symbol', 'value'),
    State('my_date_picker', 'start_date'),
    State('my_date_picker', 'end_date')])
def update_graph(n_clicks, stock_ticker, start_date, end_date):
    # since stock_ticker is now a list of symbols, create a list of traces
    traces = []
    #Update figure by indexing datetime strings.
    start_date = datetime.strptime(start_date.split(' ')[0], '%Y-%m-%d')
    end_date = datetime.strptime(end_date.split(' ')[0], '%Y-%m-%d')    
    start = start_date.strftime('%Y-%m-%d')
    end = end_date.strftime('%Y-%m-%d')
    #Filter data by selected datetime
    ciss2 = ciss.loc[start:end]
    ciss2 = hf.loc[start:end]
    traces.append({'x':ciss2.index, 'y': ciss2[stock_ticker]})
    traces.append({'x':hf.index, 'y': hf[stock_ticker]})
    fig = {
        # set data equal to traces
        'data': traces,
        'layout': {'title':'Trading Decisions:'}
    }
    return fig

if __name__ == '__main__':
    app.run_server()

SyntaxError: invalid syntax (<ipython-input-13-1d0d0d8791b2>, line 63)